# ChoiceField

```
from django.forms import Form
from django.forms import fields
from django.core.validators import ValidationError
from django.core.validators import RegexValidator
```

ChoiceField的choices无法动态获取数据库信息有2种解决方式：


In [ ]:
class ClassesForm(Form):
    name = fields.CharField(
        required=True,  # 必填字段
        error_messages={"required": "姓名不能为空！！"},  # 显示中文错误提示
        widget=widgets.TextInput(attrs={"placeholder": "姓名", "class": "form-control"}),  # 自动生成input框
    )
    # 如果直接定义成classteacher_id，，_id 的形式，这样你添加数据的时候不会时时更新，所以在下面定义一个重写的方法
    # classteacher_id = fields.ChoiceField(choices= models.UserInfo.objects.filter(ut_id = 1).values_list('id', "username"))

    classteacher_id = fields.ChoiceField(choices=[])
    def __init__(self,*args,**kwargs):   #重写init方法，时时更新
        super().__init__(*args,**kwargs)   #继承父类

        self.fields["classteacher_id"].choices = models.UserInfo.objects.filter(ut_id = 1).values_list('id', "username")

In [ ]:
from django.forms.models import ModelChoiceField  # 先导入

class ClassForm(Form):
    caption = fields.CharField(error_messages={'required':'班级名称不能为空'})
    # headmaster = fields.ChoiceField(choices=[(1,'娜娜',)])
    headmaster_id = ModelChoiceField(queryset=models.UserInfo.objects.filter(ut_id=2))

form对象常用的三个属性和方法：is_valid()；cleaned_data；errors

* cleaned_data是一个经过数据转换的字典，如把字符串的"12"转换为数字型的12.
* errors也是一个字典，字典的value是列表形式。

钩子函数：
1. clean_字段名：某个字段进行校验

    <font color="red">注意：必须有返回值；只能拿自己当前字段值；raise ValidationError("xxx")</font>
2. clean：所有字段进行校验

    <font color="red">注意：必须有返回值；可以拿所有字段的值；add_error</font>

In [ ]:
class LoginForm(Form):
    username = fields.CharField(
        required=True,  #必填字段
        min_length=3,
        max_length=16,
        error_messages={
            "required":"用户名不能为空",
            "min_length":"长度不能小于3",
            "max_length":"长度不能大于16"
        },
        widget=widgets.TextInput({"placeholder":"username","class":"form-control"})
    )
    password = fields.CharField(
        required=True,
        min_length=3,
        max_length=16,
        error_messages={
            "required": "密码不能为空",
            "min_length": "密码长度不能小于3",
            "max_length": "密码长度不能大于16",
            # "invalid":"密码格式错误"
            # error_messages的优先级高，如果写上"invalid":"密码格式错误"这个就会优先显示这个错误
        },
        widget=widgets.PasswordInput({"placeholder":"password","class":"form-control"}),
        validators=[RegexValidator("\d+","密码只能是数字")]  #可以进行正则匹配提示错误
    )

    def clean_username(self):
        user = self.cleaned_data["username"]
        is_exits = models.UserInfo.objects.filter(username=user).count()
        if not is_exits:
            raise ValidationError("用户名和密码错误")
        return user   #必须有return

# ORM的only和defer



In [ ]:
datalist = models.Userinfo.objects.all().only("name","email")  #拿到的还是一个QuerySet集合，仅仅取name和email
for item in datalist:
    print(item.id)
    print(item.name)
    print(item.pwd)   #只要表里有这个字段，一样会取到值，额外的会再发一次请求

datalist = models.Userinfo.objects.all().defer("name","email") #阻止，不取name和email
for item in datalist:
    print(item.id)
    print(item.pwd)


# form对象的data和initial属性

只有data属性调用is_valid才会进行校验工作，initial仅仅是前端展示用的。
* form = TeacherForm(data=request.POST) # 数据和规则放置一起　（添加的时候用）
* form = TeacherForm(initial={'username':obj.username,'password':obj.password,'email':obj.email})   # 显示input，并且将数据库中的默认值填写到input框中 （编辑的时候用）

# 自定义验证规则



In [ ]:
def mobile_validate(value):
    mobile_re = re.compile(r'^(13[0-9]|15[012356789]|17[678]|18[0-9]|14[57])[0-9]{8}$')
    if not mobile_re.match(value):
        raise ValidationError('手机号码格式错误')
 
 
class PublishForm(Form): 
    # 使用自定义验证规则
    phone = fields.CharField(validators=[mobile_validate, ],
                            error_messages={'required': '手机不能为空'},
                            widget=widgets.TextInput(attrs={'class': "form-control",
                                                          'placeholder': u'手机号码'}))

# ModelForm

通常在Django项目中，我们编写的大部分都是与Django 的模型紧密映射的表单。 举个例子，你也许会有个Book 模型，并且你还想创建一个form表单用来添加和编辑书籍信息到这个模型中。 在这种情况下，在form表单中定义字段将是冗余的，因为我们已经在模型中定义了那些字段。

基于这个原因，Django 提供一个辅助类来让我们可以从Django 的模型创建Form，这就是ModelForm。

与普通的Form表单验证类型类似，ModelForm表单的验证在调用is_valid() 或访问errors 属性时隐式调用。

我们可以像使用Form类一样自定义局部钩子方法和全局钩子方法来实现自定义的校验规则。

如果我们不重写具体字段并设置validators属性的化，ModelForm是按照模型中字段的validators来校验的。

In [ ]:
class BookForm(forms.ModelForm):

    class Meta:
        model = models.Book
        fields = "__all__"
        labels = {
            "title": "书名",
            "price": "价格"
        }
        widgets = {
            "password": forms.widgets.PasswordInput(attrs={"class": "c1"}),
        }

## Meta下常用参数

```
model = models.Book  # 对应的Model中的类
fields = "__all__"  # 字段，如果是__all__, 就是表示列出所有的字段, 也可以是一个列表
exclude = None  # 排除的字段, 可以是一个列表
labels = None  # 提示信息
help_texts = None  # 帮助提示信息
widgets = None  # 自定义插件
error_messages = None  # 自定义错误信息
```

## save方法

每个ModelForm还具有一个save()方法。这个方法根据表单绑定的数据创建并能把对象保存进入数据库。ModelForm可以接受现有的模型实例作为关键字参数instance, 如果提供此参数, 则save()将根据提交的数据来更新该实例。如果没有提instance参数, save()将创建模型的一个新实例。

In [ ]:
from myapp.models import Book
from myapp.forms import BookForm

# 根据POST数据创建一个新的form对象
form_obj = BookForm(request.POST)

# 创建书籍对象
new_ book = form_obj.save()

# 基于一个书籍对象创建form对象
edit_obj = Book.objects.get(id=1)
# 使用POST提交的数据更新书籍对象
form_obj = BookForm(request.POST, instance=edit_obj)
form_obj.save()